# Logistic Regression Problem Formulation

### 1. Goal
> Predict whether a car is **American-made** (`origin = 1`) or **foreign-made** (`origin = 2 or 3`) using its technical specifications.

### 2. Problem Statement
> We want to classify cars as either American or foreign based on features such as `cylinders`, `displacement`, `horsepower`, `weight`, and `acceleration`. This is a binary classification task where the target variable is derived from the `origin` column.

### 3. Target Variable
- Original column: `origin`
- Transformed binary label:
  - `1` → American-made
  - `0` → Foreign-made (Europe or Japan)

### 4. Input Features
- `cylinders`
- `displacement`
- `horsepower`
- `weight`
- `acceleration`

These reflect that engine size, power, and design choices may differ by region.

### 5. Hypotheses
- **Null Hypothesis (H₀)**: None of the selected features significantly influence whether a car is American-made. All coefficients are zero.
- **Alternative Hypothesis (H₁)**: At least one of the selected features significantly influences the likelihood that a car is American-made.

### 6. Example Input
```csv
cylinders, displacement, horsepower, weight, acceleration
4,113,95,2372,15
```

### 7. Predicted Output
```csv
0 → Foreign-made
```

## The expected approach involves:

1. Formulating the Model: Defining the linear combination of features and the sigmoid function.

2. Using Gradient Descent: This is the standard optimization algorithm used to train the Logistic Regression model (just as it is used to train DNNs).

3. Using the Sigmoid Function: This is the link function that maps the linear output to a probability between 0 and 1.

4. Using the Log-Loss (Cross-Entropy) Function: This is the standard cost function for classification tasks like this.

In [ ]:
import pandas as pd
